In [1]:
"""uncomment it in case gynmasium is not present"""

# %pip install gymnasium[all]
# %pip install shimmy
# %pip install "gymnasium[atari, accept-rom-license]"

"""uncomment it to check if Pong-v0 and others are present or not"""
# # To Check if all modules are working fine 
# import ale_py
# import gymnasium
# import shimmy

# import gymnasium as gym

# req = set(['MountainCar-v0','Pong-v0','CartPole-v0','LunarLander-v2'])
# keys = set(gym.envs.registry.keys())
# for key in req:

#   if(keys.intersection({key}) == set({})):
#       print(key +' not found')   


import gymnasium as gym
import numpy as np
import shimmy #it is compulsory to import
import ale_py #it's too

env_1 = gym.make("Pong-v0")
print('\n Environment : Pong-v0')
 # Print state and action space
print("State space:", env_1.observation_space)
print("Action space:", env_1.action_space)
state, _ = env_1.reset()
done = False
total_reward = 0
step_count = 0
rew_1 = {}
print("Running random agent...")


while not done:
    action = env_1.action_space.sample()  # Random action
    state, reward, done, _, _ = env_1.step(action)
    total_reward += reward
    if reward not in rew_1:
        rew_1[reward] = 0
    rew_1[reward] += 1
    step_count += 1

print(f"Total reward after {step_count} steps: {total_reward}\n")

# Close the environment
env_1.close()
print(f'These are the Rewards(and respective frequencies) after experienting with Random Agent : \n {rew_1}')


c:\Users\charu\anaconda3\Lib\site-packages\gymnasium\envs\registration.py:517: DeprecationWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(



 Environment : Pong-v0
State space: Box(0, 255, (210, 160, 3), uint8)
Action space: Discrete(6)
Running random agent...
Total reward after 1058 steps: -21.0

These are the Rewards(and respective frequencies) after experienting with Random Agent : 
 {0.0: 1037, -1.0: 21}


In [2]:
import random

env_2 = gym.make("Pong-ram-v0")
print('\n Environment : Pong-v0')
 # Print state and action space
print("State space:", env_2.observation_space)
print("Action space:", env_2.action_space)
state, _ = env_2.reset()
done = False
total_reward = 0
step_count = 0
rew_2={}
allowed_actions = [0,2,3]
print("Running random agent...")


while not done:
    action = random.choice(allowed_actions)  # Random action
    state, reward, done, _, _ = env_2.step(action)
    total_reward += reward
    if reward not in rew_2:
        rew_2[reward] = 0
    rew_2[reward] += 1
    step_count += 1

print(f"Total reward after {step_count} steps: {total_reward}\n")

# Close the environment
env_2.close()

print(f'These are the Rewards(and their frequencies) after experienting with Random Agent \n {rew_2}')


 Environment : Pong-v0
State space: Box(0, 255, (128,), uint8)
Action space: Discrete(6)
Running random agent...


c:\Users\charu\anaconda3\Lib\site-packages\gymnasium\envs\registration.py:517: DeprecationWarning: WARN: The environment Pong-ram-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(


Total reward after 1108 steps: -21.0

These are the Rewards(and their frequencies) after experienting with Random Agent 
 {0.0: 1087, -1.0: 21}


In [3]:
import cv2

# import time
import json
import random
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm

from collections import deque

ENVIRONMENT = "Pong-v0"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# DEVICE = 'cpu'

SAVE_MODELS = True  # Save models to file so you can test later
MODEL_PATH = "./pong-cnn-"  # Models path for saving or loading
SAVE_MODEL_INTERVAL = 50  # Save models at every X epoch
TRAIN_MODEL = True  # Train model while playing (Make it False when testing a model)

LOAD_MODEL_FROM_FILE = True  # Load model from file
LOAD_FILE_EPISODE = 500  # Load Xth episode from file

BATCH_SIZE = 32  # Minibatch size that select randomly from mem for train nets
MAX_EPISODE = 1500  # Max episode
MAX_STEP = 2000  # Max step size for one episode

MAX_MEMORY_LEN = 20000  # Max memory len
MIN_MEMORY_LEN = 1000  # Min memory len before start train -----------------------------

GAMMA = 0.99  # Discount rate
ALPHA = 0.00025  # Learning rate
EPSILON_DECAY = 0.995  # Epsilon decay rate by step


RENDER_GAME_WINDOW = False  # Opens a new window to render the game (Won't work on colab default)

In [4]:
import torch
import torch.nn as nn

class DQNPong(nn.Module):
    def __init__(self, action_space):
        super(DQNPong, self).__init__()
        self.conv = nn.Sequential(
            # nn.Conv2d(4, 32, kernel_size=8, stride=4),
            # nn.LeakyReLU(),
            # nn.BatchNorm2d(32),
            # nn.Conv2d(32, 64, kernel_size=4, stride=2),
            # nn.LeakyReLU(),
            # nn.BatchNorm2d(64),
            # nn.Conv2d(64, 64, kernel_size=3, stride=1),
            # nn.LeakyReLU()
            nn.Conv2d(4,16,8,4),
            nn.ReLU(),
            nn.Conv2d(16,32,4,2),
            nn.ReLU(),
        )
        
        self.fc = nn.Sequential(
            nn.Linear(32*9*9, 256),  
            nn.ReLU(),
            nn.Linear(256, action_space)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor before passing it to fully connected layers
        x = self.fc(x)
        return x


In [5]:
class Agent:
    def __init__(self, environment):
        """
        Hyperparameters definition for Agent
        """
        # State size for breakout env. SS images (210, 160, 3). Used as input size in network
        self.state_size_h = environment.observation_space.shape[0]
        self.state_size_w = environment.observation_space.shape[1]
        self.state_size_c = environment.observation_space.shape[2]

        # Activation size for breakout env. Used as output size in network
        self.action_size = environment.action_space.n

        # Image pre process params
        self.target_h = 84  # Height after process
        self.target_w = 84  # Width after process

        self.crop_dim = [20, self.state_size_h, 0, self.state_size_w]  # Cut 20 px from top to get rid of the score table

        # Trust rate to our experiences
        self.gamma = GAMMA  # Discount coef for future predictions
        self.alpha = ALPHA  # Learning Rate

        # After many experinces epsilon will be 0.05
        # So we will do less Explore more Exploit
        self.epsilon = 1  # Explore or Exploit
        self.epsilon_decay = EPSILON_DECAY  # Adaptive Epsilon Decay Rate
        self.epsilon_minimum = 0.05  # Minimum for Explore

        # Deque holds replay mem.
        self.memory = deque(maxlen=MAX_MEMORY_LEN)

        # Create two model for DDQN algorithm
        self.online_model = DQNPong(self.action_size).to(DEVICE) #------------change here
        self.target_model = DQNPong(self.action_size).to(DEVICE) #------------change here
        self.target_model.load_state_dict(self.online_model.state_dict())
        self.target_model.eval()

        # Adam used as optimizer
        self.optimizer = optim.Adam(self.online_model.parameters(), lr=self.alpha,weight_decay=1e-5)

    def preProcess(self, image):
        """
        Process image crop resize, grayscale and normalize the images
        """

        # print(type(image))
        # image = image[0]
        frame = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)  # To grayscale
        frame = frame[self.crop_dim[0]:self.crop_dim[1], self.crop_dim[2]:self.crop_dim[3]]  # Cut 20 px from top
        frame = cv2.resize(frame, (self.target_w, self.target_h))  # Resize
        frame = frame.reshape(self.target_w, self.target_h) / 255  # Normalize

        return frame

    def act(self, state):
        """
        Get state and do action
        Two option can be selectedd if explore select random action
        if exploit ask nnet for action
        """

        act_protocol = 'Explore' if random.uniform(0, 1) <= self.epsilon else 'Exploit'

        if act_protocol == 'Explore':
            action = random.randrange(self.action_size)
        else:
            with torch.no_grad():
                state = torch.tensor(state, dtype=torch.float, device=DEVICE).unsqueeze(0)
                q_values = self.online_model.forward(state)  # (1, action_size)
                action = torch.argmax(q_values).item()  # Returns the indices of the maximum value of all elements

        return action

    def train(self):
        """
        Train neural nets with replay memory
        returns loss and max_q val predicted from online_net
        """
        if len(self.memory) < MIN_MEMORY_LEN:
            loss, max_q = [0, 0]
            return loss, max_q
        # We get out minibatch and turn it to numpy array
        state, action, reward, next_state, done = zip(*random.sample(self.memory, BATCH_SIZE))

        # Concat batches in one array
        # (np.arr, np.arr) ==> np.BIGarr
        state = np.concatenate(state)
        next_state = np.concatenate(next_state)

        # Convert them to tensors
        state = torch.tensor(state, dtype=torch.float, device=DEVICE)
        next_state = torch.tensor(next_state, dtype=torch.float, device=DEVICE)
        action = torch.tensor(action, dtype=torch.long, device=DEVICE)
        reward = torch.tensor(reward, dtype=torch.float, device=DEVICE)
        done = torch.tensor(done, dtype=torch.float, device=DEVICE)

        # Make predictions
        state_q_values = self.online_model(state)
        next_states_q_values = self.online_model(next_state)
        next_states_target_q_values = self.target_model(next_state)

        # Find selected action's q_value
        selected_q_value = state_q_values.gather(1, action.unsqueeze(1)).squeeze(1)
        # Get indice of the max value of next_states_q_values
        # Use that indice to get a q_value from next_states_target_q_values
        # We use greedy for policy So it called off-policy
        next_states_target_q_value = next_states_target_q_values.gather(1, next_states_q_values.max(1)[1].unsqueeze(1)).squeeze(1)
        # Use Bellman function to find expected q value
        expected_q_value = reward + self.gamma * next_states_target_q_value * (1 - done)

        # Calc loss with expected_q_value and q_value
        # loss = (selected_q_value - expected_q_value.detach()).pow(2).mean()
        criterion = torch.nn.MSELoss()
        loss = criterion(selected_q_value, expected_q_value)


        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        return loss, torch.max(state_q_values).item()

    def storeResults(self, state, action, reward, nextState, done):
        """
        Store every result to memory
        """
        self.memory.append([state[None, :], action, reward, nextState[None, :], done])

    def adaptiveEpsilon(self):
        """
        Adaptive Epsilon means every step
        we decrease the epsilon so we do less Explore
        """
        if self.epsilon > self.epsilon_minimum:
            self.epsilon *= self.epsilon_decay

In [ ]:
environment = gym.make(ENVIRONMENT)  # Get env
agent = Agent(environment)  # Create Agent

if LOAD_MODEL_FROM_FILE:
    agent.online_model.load_state_dict(torch.load(MODEL_PATH+str(LOAD_FILE_EPISODE)+".pth"))

    with open(MODEL_PATH+str(LOAD_FILE_EPISODE)+'.json') as outfile:
        param = json.load(outfile)
        agent.epsilon = param.get('epsilon')

    startEpisode = LOAD_FILE_EPISODE + 1

else:
    startEpisode = 1

last_100_ep_reward = deque(maxlen=100)  # Last 100 episode rewards
total_step = 1  # Cumulkative sum of all steps in episodes
for episode in tqdm(range(startEpisode, MAX_EPISODE),colour='green'):

    # startTime = time.time()  # Keep time
    state,_ = environment.reset()  # Reset env
    
    state = agent.preProcess(state)  # Process image

    # Stack state . Every state contains 4 time contionusly frames
    # We stack frames like 4 channel image
    state = np.stack((state, state, state, state))

    total_max_q_val = 0  # Total max q vals
    total_reward = 0  # Total reward for each episode
    total_loss = 0  # Total loss for each episode
    done = False
    step = 0
    while(not done):

        if RENDER_GAME_WINDOW:
            environment.render()  # Show state visually

        step+=1
        # Select and perform an action
        action = agent.act(state)  # Act
        next_state, reward, done,_,_ = environment.step(action)  # Observe
        # print(next_state.shape)
        next_state = agent.preProcess(next_state)  # Process image

        # Stack state . Every state contains 4 time contionusly frames
        # We stack frames like 4 channel image
        next_state = np.stack((next_state, state[0], state[1], state[2]))

        # Store the transition in memory
        agent.storeResults(state, action, reward, next_state, done)  # Store to mem

        # Move to the next state
        state = next_state  # Update state

        if TRAIN_MODEL:
            # Perform one step of the optimization (on the target network)
            loss, max_q_val = agent.train()  # Train with random BATCH_SIZE state taken from mem
        else:
            loss, max_q_val = [0, 0]

        total_loss += loss
        total_max_q_val += max_q_val
        total_reward += reward
        total_step += 1
        if total_step % 1000 == 0:
            agent.adaptiveEpsilon()  # Decrase epsilon

        if TRAIN_MODEL and step%200==0:
                agent.target_model.load_state_dict(agent.online_model.state_dict())  # Update target model
        
        if done:  # Episode completed
            
            epsilonDict = {'epsilon': agent.epsilon}  # Create epsilon dict to save model as file

            if SAVE_MODELS and episode % SAVE_MODEL_INTERVAL == 0:  # Save model as file
                weightsPath = MODEL_PATH + str(episode) + '.pth'
                epsilonPath = MODEL_PATH + str(episode) + '.json'

                torch.save(agent.online_model.state_dict(), weightsPath)
                with open(epsilonPath, 'w') as outfile:
                    json.dump(epsilonDict, outfile)

            last_100_ep_reward.append(total_reward)
            avg_max_q_val = total_max_q_val / step

            outStr = "Episode:{}  Reward:{:.2f} Loss:{:.2f} Last_100_Avg_Rew:{:.3f} Avg_Max_Q:{:.3f} Epsilon:{:.2f} Step:{} CStep:{}".format(
                episode, total_reward, total_loss, np.mean(last_100_ep_reward), avg_max_q_val, agent.epsilon,  step, total_step
            )
            if(episode%100==0):
                print(outStr)
            if episode%10==0:
                if SAVE_MODELS:
                    outputPath = MODEL_PATH + "out" + '.csv'  # Save outStr to file
                    with open(outputPath, 'a') as outfile:
                        outfile.write(f'{episode},{total_reward},{total_loss},{np.mean(last_100_ep_reward)}, {avg_max_q_val}, {agent.epsilon},  {step}, {total_step}'+"\n")

                break

c:\Users\charu\anaconda3\Lib\site-packages\gymnasium\envs\registration.py:517: DeprecationWarning: WARN: The environment Pong-v0 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
C:\Users\charu\AppData\Local\Temp\ipykernel_10720\2318296940.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for

Episode:600  Reward:-17.00 Loss:13.33 Last_100_Avg_Rew:-16.960 Avg_Max_Q:1.347 Epsilon:0.05 Step:2599 CStep:216904


 20%|██        | 200/999 [1:37:15<6:28:23, 29.17s/it]

Episode:700  Reward:-16.00 Loss:14.89 Last_100_Avg_Rew:-16.500 Avg_Max_Q:1.741 Epsilon:0.05 Step:2414 CStep:450255


 30%|███       | 300/999 [2:27:18<5:57:49, 30.71s/it]

...

 80%|████████  | 800/999 [6:40:37<1:42:19, 30.85s/it]

Episode:1300  Reward:-18.00 Loss:11.79 Last_100_Avg_Rew:-15.680 Avg_Max_Q:1.743 Epsilon:0.05 Step:2100 CStep:1881674


 90%|█████████ | 900/999 [7:31:18<52:48, 32.00s/it]  

Episode:1400  Reward:-16.00 Loss:15.39 Last_100_Avg_Rew:-15.680 Avg_Max_Q:1.777 Epsilon:0.05 Step:2982 CStep:2123682


100%|██████████| 999/999 [8:22:23<00:00, 30.17s/it]
